# Long Short-term Memory for Text Generation

This notebook uses LSTM neural network to generate text from Nietzsche's writings.

In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import random
import sys
import io


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import get_file

## Dataset

### Get the data
Nietzsche's writing dataset is available online. The following code download the dataset.

In [60]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

### Visualize data

In [63]:
print('corpus length:', len(text))

corpus length: 600893


In [65]:
print(text[10:513])

supposing that truth is a woman--what then? is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to truth, have been unskilled and unseemly methods for
winning a woman? certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--if,
indeed, it stands at all!


In [67]:
chars = sorted(list(set(text)))
# total nomber of characters
print('total chars:', len(chars))

total chars: 57


### Clean data

We cut the text in sequences of maxlen characters with a jump size of 3.
The features for each example is a matrix of size maxlen*num of chars.
The label for each example is a vector of size num of chars, which represents the next character.

In [70]:
# create (character, index) and (index, character) dictionary
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [72]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


### Vecterization **This vectorization procedure includes a one-hot-encoding for each word, and therefore, an Embedding layer should not be used. If you want to use an Embedding layer. You will need to revise this block. **

In [75]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


## The model

### Build the model - fill in this box

we need a recurrent layer with input shape (maxlen, len(chars)) and a dense layer with output size  len(chars)

In [77]:
x.shape

(200285, 40, 57)

In [79]:
model = keras.Sequential()
model.add(layers.LSTM(256, return_sequences=True, input_shape=(maxlen, len(chars)))) 
model.add(layers.LSTM(256)) 
model.add(layers.Dense(len(chars), activation='softmax'))  

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',  
              optimizer=optimizer)

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [82]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 40, 256)        │       321,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 57)             │        14,649 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 861,497 (3.29 MB)

 Trainable params: 861,497 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

### Train the model

In [85]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [87]:
class PrintLoss(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, _):
        # Function invoked at end of each epoch. Prints generated text.
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.5, 1.0]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [89]:
EPOCHS = 10
BATCH = 128

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(x, y,
                    batch_size = BATCH,
                    epochs = EPOCHS,
                    validation_split = 0.2,
                    verbose = 1,
                    callbacks = [early_stop, PrintLoss()])

Epoch 1/10
1252/1252 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 2.7239
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: " but of a very limited kind in compariso"
 but of a very limited kind in comparisouled the mong the lought the rething prestion of the the will the most the expinate the cooprest in sidsouss, and werther for ever to it is as sould the selpess and and for it is the loving the porpoustiont of the gore is suthing at indous of the serbent and peremon aly centerstion of the conced in the sent the grees and esting it the soplestenter of the aster in the prising the per of perate to p
----- diversity: 1.0
----- Generating with seed: " but of a very limited kind in compariso"
 but of a very limited kind in compariso bither,
ond pracianter--verent of zevee and mesentaund, a illoots, me.

259 giss it ( p lnoud is thelescel.

92 thet ere
tonk in smins muthigh- fetun
bsounianmer. "vury coms., ane an one mofe sis peitated no- to calqaocco his 